In [1]:
import argparse
import yaml

from cxrclip import build_model
from cxrclip.loss import build_loss
from data import load_tokenizer
from cxrclip.downstream import CxrDecoderTrainer
from data.loaders import CxrDataLoader

/Users/tudormihaita/PycharmProjects/clip-cxr-report-gen/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/tudormihaita/PycharmProjects/clip-cxr-report-gen/.venv/lib/python3.9/site-packages/albumentations/__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.7' (you have '2.0.6'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/tudormihaita/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/tudormihaita/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading packa

In [2]:
tokenizer_config = yaml.load(open("../configs/tokenizer/radiology_bart.yaml"), Loader=yaml.FullLoader)
model_config = yaml.load(open("../configs/model/bart_report_gen.yaml"), Loader=yaml.FullLoader)
loss_config = yaml.load(open("../configs/loss/language_modeling.yaml"), Loader=yaml.FullLoader)

tokenizer = load_tokenizer(**tokenizer_config)
model = build_model(model_config, loss_config, tokenizer, use_custom=True)
loss_fn = build_loss(loss_config)

/Users/tudormihaita/PycharmProjects/clip-cxr-report-gen/.venv/lib/python3.9/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
2025-05-21 01:37:26,565 - INFO - Loading pre-trained image encoder from checkpoint
Some weights of the model checkpoint at microsoft/swin-tiny-patch4-window7-224 were not used when initializing SwinModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing SwinModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing SwinModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertFo

KeyError: 'model'

In [3]:
transform_config = yaml.load(open("../configs/transform/clahe.yaml"), Loader=yaml.FullLoader)
train_config = yaml.load(open("../configs/train/report_gen.yaml"), Loader=yaml.FullLoader)

args = argparse.Namespace(
    dataset_name='mimic-cxr-mvs',
    batch_size=10,
    max_length=train_config["max_length"],
    image_size=train_config["image_size"],
    num_workers=0,
    drop_last=True,
    use_minio=False
)

In [4]:
dataloaders = {
    "train": CxrDataLoader(
        args,
        split='train',
        transform_config=transform_config,
        tokenizer=tokenizer,
    ),
    "val": CxrDataLoader(
        args,
        split='val',
        transform_config=transform_config,
        tokenizer=tokenizer,
    ),
    "test": CxrDataLoader(
        args,
        split='test',
        transform_config=transform_config,
        tokenizer=tokenizer,
    ),
}

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/tudormihaita/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/tudormihaita/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/tudormihaita/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [5]:
trainer = CxrDecoderTrainer(
    model=model,
    config=train_config,
    loss_fn=loss_fn,
    train_loader=dataloaders["train"],
    val_loader=dataloaders["val"],
    mixed_precision=False
)

2025-05-21 01:09:40,161 - INFO - Total parameters: 317,582,714
2025-05-21 01:09:40,161 - INFO - Trainable parameters: 317,582,714 (100.00%)


In [6]:
trainer.train()

2025-05-21 01:09:43,455 - INFO - Starting training
2025-05-21 01:09:43,455 - INFO - Starting epoch 1/20
Epoch 1/20:   0%|          | 0/21342 [00:00<?, ?it/s]


AttributeError: 'CxrCLIPDecoder' object has no attribute 'encode_imnage'